numpy, pandas, difflib, cosine_similarity, feature_extracton: TfidfVectorizer

In [72]:
import numpy as np
import pandas as pd
import difflib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer 

* Load data
* Chọn ra đặc trưng
* Thay thế nullvalue bằng nullstring với mỗi cột đặc trưng
* Kết hợp tất cả các đặc trưng
* Chuyển từ dữ liệu text về vector đặc trưng
  * Sử dụng TfidfVectorizer
    * TfidfVectorizer sẽ tính độ quan trọng của từ trong tài liệu theo một từ điển:
        * Chia văn bản thành các tokens
        * Xây dựng từ điển chứa tất cả các từ trong tập dữ liệu, mỗi từ được gán chỉ số
    * TfidfVectorizer.fit_transform sẽ trả về ma trận thưa có định dạng: 
        (văn bản, index của từ trong từ điển) giá trị TF-IDF
* Đánh giá độ tương đồng của mỗi phim với nhau

* Use Case
  * Người dùng nhập tên phim vào
  * Hệ thống tìm kiếm kết quả dựa trên phim người dùng nhập vào (get_close_match)
  



# Thu thập và tiền xử lý dữ liệu

## Thu thập dữ liệu

In [73]:
movies_data = pd.read_csv('movies.csv')

### Đưa ra 5 dòng đầu của tập dữ liệu

In [74]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


Ý nghĩa các thuộc tính:

* budget: ngân sách
* genres: thể loại
* homepage: trang web của phim
* id: id phim trong IMDB

### In ra số hàng, số cột của dữ liệu

In [75]:
movies_data.shape

(4803, 24)

In [76]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

### Thay thế các giá trị rỗng thành xâu rỗng

In [77]:
for col in movies_data.columns:
    movies_data[col] = movies_data[col].fillna('')

# Chọn ra đặc trưng cho các bộ phim

Các thuộc tính sẽ được chọn:
* genres: thể loại
* keywords: từ khóa
* cast: diễn viên
* director: đạo diễn

In [78]:
selected_features = ['genres', 'keywords', 'cast', 'director']
print(selected_features)

['genres', 'keywords', 'cast', 'director']


# Kết hợp các đặc trưng

In [79]:
combined_features = movies_data['genres'] + ' ' + movies_data['keywords'] + ' ' + movies_data['cast'] + ' ' + movies_data['director']
print(combined_features)


0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  Edward Burns Kerry Bish\u00e9 ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      Daniel Henney Eliza Coupe Bill Paxton Alan R...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


# Trích xuất vector đặc trưng từ các đặc trưng của dữ liệu

Xây dựng vector đặc trưng bằng TfIdfVectorizer

In [80]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 97547 stored elements and shape (4803, 14845)>
  Coords	Values
  (0, 86)	0.08508837387249067
  (0, 144)	0.09834166706222157
  (0, 4435)	0.12020924576100474
  (0, 11745)	0.11214224678270057
  (0, 4566)	0.1122735385607623
  (0, 3115)	0.23133291228677105
  (0, 2616)	0.24276191286779358
  (0, 4886)	0.19838040431463286
  (0, 12386)	0.376936512374071
  (0, 14235)	0.1389277198420819
  (0, 2755)	0.2699167358611062
  (0, 12299)	0.23546951253664175
  (0, 11517)	0.16271392202818094
  (0, 14561)	0.25567659458637093
  (0, 14820)	0.2184182305681937
  (0, 11490)	0.23546951253664175
  (0, 12134)	0.22272408661423276
  (0, 14291)	0.21458309577216156
  (0, 12567)	0.16383787425669902
  (0, 7496)	0.24557305356150633
  (0, 8831)	0.17365703713079808
  (0, 11217)	0.21004511389940914
  (0, 6542)	0.12250886401691784
  (0, 2061)	0.18678214793744197
  (1, 86)	0.07336074232004275
  :	:
  (4801, 10069)	0.24105780118392384
  (4801, 5844)	0.322725945886879

Tính độ tương đồng cho các vector đặc trưng của mỗi dữ liệu

In [81]:
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.02703888 0.04479062 ... 0.         0.         0.        ]
 [0.02703888 1.         0.01505199 ... 0.01653831 0.         0.        ]
 [0.04479062 0.01505199 1.         ... 0.         0.06576044 0.        ]
 ...
 [0.         0.01653831 0.         ... 1.         0.         0.02554794]
 [0.         0.         0.06576044 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02554794 0.         1.        ]]


In [82]:
print(similarity.shape)

(4803, 4803)


# Xây dựng kịch bản cho UseCase của người dùng

## Người dùng nhập vào tên phim

In [83]:
movie_name = input('Nhap vao ten phim: ')
print(movie_name)

Avengers


## Lấy ra tên của tất cả các phim trong hệ thống

In [84]:
all_movie_names = movies_data['title'].tolist()
print(all_movie_names)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

## Đưa ra tên bộ phim gần nhất với tên phim nhập vào của người dùng

In [85]:
close_matches = difflib.get_close_matches(movie_name, all_movie_names)
print(close_matches)

['The Avengers', 'Swingers']


In [86]:
close_match = close_matches[0]
print(close_match)

The Avengers


## Lấy ra index của bộ phim đó

In [87]:
movie_index = movies_data[movies_data['title'] == close_match]['index'].values[0]
print(movie_index)

16


## Lấy ra các bộ phim tương đồng với bộ phim này

In [88]:
similarity_score = list(enumerate(similarity[movie_index]))
print(similarity_score)

[(0, np.float64(0.0407419246426015)), (1, np.float64(0.01411250613093922)), (2, np.float64(0.05117648021547665)), (3, np.float64(0.007642999700476072)), (4, np.float64(0.07455052946881038)), (5, np.float64(0.016148262719449174)), (6, np.float64(0.0)), (7, np.float64(0.8911938130734699)), (8, np.float64(0.009650831683202275)), (9, np.float64(0.19710821613866303)), (10, np.float64(0.0832577082268175)), (11, np.float64(0.01586003259899002)), (12, np.float64(0.01594030805327245)), (13, np.float64(0.014874183222413628)), (14, np.float64(0.22620315111650013)), (15, np.float64(0.03904706068461799)), (16, np.float64(1.0000000000000002)), (17, np.float64(0.016375929387939452)), (18, np.float64(0.028976764586895083)), (19, np.float64(0.01644538526565378)), (20, np.float64(0.13077749457072868)), (21, np.float64(0.03688387970252563)), (22, np.float64(0.009866066198810773)), (23, np.float64(0.052321928089969716)), (24, np.float64(0.015330135068839169)), (25, np.float64(0.0)), (26, np.float64(0.4786

In [89]:
len(similarity_score)

4803

### Sắp xếp độ tương đồng

In [90]:
sorted_similar_movies = sorted(similarity_score, key= lambda movie_info: movie_info[1], reverse= True)
print(sorted_similar_movies)

[(16, np.float64(1.0000000000000002)), (7, np.float64(0.8911938130734699)), (79, np.float64(0.5127916691631869)), (85, np.float64(0.5100942688795995)), (26, np.float64(0.47867997472672236)), (126, np.float64(0.37650021139406553)), (174, np.float64(0.35661453938396437)), (511, np.float64(0.33937286460502064)), (203, np.float64(0.33826433149938856)), (182, np.float64(0.3334296779271291)), (64, np.float64(0.3291939430370663)), (129, np.float64(0.3172500958254604)), (46, np.float64(0.3065880277216466)), (788, np.float64(0.30084106351824697)), (169, np.float64(0.28981319607708067)), (33, np.float64(0.2886767530201519)), (68, np.float64(0.2819490509837047)), (38, np.float64(0.275092332113928)), (31, np.float64(0.26755497807569845)), (4759, np.float64(0.2567212451029577)), (101, np.float64(0.2293210643104345)), (870, np.float64(0.22927646464339796)), (782, np.float64(0.22927301891071505)), (14, np.float64(0.22620315111650013)), (9, np.float64(0.19710821613866303)), (94, np.float64(0.194775818

### In ra tên những bộ phim tương đồng

In [91]:
print('Top 5 phim duoc goi y: \n')
top = 1
for movie_info in sorted_similar_movies:
    if top > 5:
        break
    index = movie_info[0]
    title = movies_data[movies_data.index == index]['title'].values[0]
    print(f"{top}. {title}")
    top += 1

Top 5 phim duoc goi y: 

1. The Avengers
2. Avengers: Age of Ultron
3. Iron Man 2
4. Captain America: The Winter Soldier
5. Captain America: Civil War


# Hệ gợi ý đơn giản

In [93]:
all_movie_names = movies_data['title'].tolist()

movie_name = input('Nhap vao ten phim: ')

close_matches = difflib.get_close_matches(movie_name, all_movie_names)
close_match = close_matches[0]

movie_index = movies_data[movies_data['title'] == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[movie_index]))
sorted_similar_movies = sorted(similarity_score, key= lambda movie_info: movie_info[1], reverse= True)

print('Top 5 phim duoc goi y: \n')
top = 1
for movie_info in sorted_similar_movies:
    if top > 5:
        break
    index = movie_info[0]
    if index == movie_index:
        continue
    title = movies_data[movies_data.index == index]['title'].values[0]
    print(f"{top}. {title}")
    top += 1


Top 5 phim duoc goi y: 

1. Guardians of the Galaxy
2. Aliens
3. Alien
4. Star Trek Into Darkness
5. Galaxy Quest
